In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("github_data").master("local[*]").getOrCreate()

In [4]:
df = spark.read.json(r"C:\data\raw\github\uncompressed\2015\1\2015-01-01-5.json")

In [5]:
df.printSchema()

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- pull_request: struct (nullable = true)
 |    |    |    |    |-- href: strin

In [6]:
df.createOrReplaceTempView("allcols")

In [7]:
issues_df = spark.sql("""
SELECT
payload.issue.id issue_id,
repo.id repo_id,
payload.issue.title issue_title
FROM allcols 
where type = 'IssuesEvent' and payload.action = 'opened'
""")


In [8]:
pr_df = spark.sql("""
SELECT 
payload.pull_request.id pr_id,
repo.id repo_id,
payload.pull_request.title title
FROM allcols 
where type = 'PullRequestEvent' and payload.action = 'opened'
""")

In [9]:
starred_df = spark.sql("""
SELECT 
distinct
actor.id login_id,
repo.id repo_id
FROM allcols
where type = 'WatchEvent' and payload.action = 'started'
""")


In [10]:
forked_df = spark.sql("""
SELECT
distinct
actor.id login_id, 
repo.id repo_id
FROM allcols 
WHERE type = 'ForkEvent'
""")

forked_df.where(forked_df.repo_id=="26730195").count()

0

In [11]:
project_df = spark.sql("""
SELECT DISTINCT 
repo.id repo_id, 
repo.name repo_name
FROM allcols 
""")
project_df.where(project_df.repo_id=="26730195").count()

1

In [12]:
project_df.createOrReplaceTempView("project_df")
forked_df.createOrReplaceTempView("forked_df")
starred_df.createOrReplaceTempView("starred_df")
pr_df.createOrReplaceTempView("pr_df")
issues_df.createOrReplaceTempView("issues_df")

In [67]:
spark.sql("""
SELECT 
p.repo_id, 
p.repo_name, 
count(pr.pr_id),
count(i.issue_id),
count(s.login_id),
count(f.actor_id)
FROM project_df p
LEFT JOIN forked_df f on f.repo_id = p.repo_id
LEFT JOIN starred_df s on s.repo_id = p.repo_id
LEFT JOIN pr_df pr on pr.repo_id = p.repo_id
LEFT JOIN issues_df i on i.repo_id = p.repo_id 
GROUP BY p.repo_id, p.repo_name
ORDER BY count(pr_id) DESC
""").toPandas()

AnalysisException: "cannot resolve '`f.actor_id`' given input columns: [s.repo_id, i.repo_id, pr.repo_id, pr.title, p.repo_id, f.repo_id, i.issue_title, i.issue_id, p.repo_name, s.login_id, pr.pr_id, f.login_id]; line 8 pos 6;\n'Sort ['count('pr_id) DESC NULLS LAST], true\n+- 'Aggregate [repo_id#628L, repo_name#629], [repo_id#628L, repo_name#629, count(pr_id#588L) AS count(pr_id)#645L, count(issue_id#578L) AS count(issue_id)#646L, count(login_id#605L) AS count(login_id)#647L, unresolvedalias('count('f.actor_id), None)]\n   +- Join LeftOuter, (repo_id#579L = repo_id#628L)\n      :- Join LeftOuter, (repo_id#589L = repo_id#628L)\n      :  :- Join LeftOuter, (repo_id#606L = repo_id#628L)\n      :  :  :- Join LeftOuter, (repo_id#613L = repo_id#628L)\n      :  :  :  :- SubqueryAlias `p`\n      :  :  :  :  +- SubqueryAlias `project_df`\n      :  :  :  :     +- Distinct\n      :  :  :  :        +- Project [repo#568.id AS repo_id#628L, repo#568.name AS repo_name#629]\n      :  :  :  :          

In [16]:
result = spark.sql("""
WITH forked AS (
SELECT 
p.repo_id repo_id, 
p.repo_name repo_name,
count(1) number_of_unique_forks
FROM project_df as p
JOIN forked_df f on p.repo_id = f.repo_id
GROUP BY p.repo_id, p.repo_name
),
issues AS (
SELECT 
p.repo_id repo_id, 
p.repo_name repo_name,
count(1) number_of_issues
FROM project_df as p
JOIN issues_df i on p.repo_id = i.repo_id
GROUP BY p.repo_id, p.repo_name
),
starred AS (
SELECT 
p.repo_id repo_id, 
p.repo_name repo_name,
count(1) number_of_stars
FROM project_df as p
JOIN starred_df s on p.repo_id = s.repo_id
GROUP BY p.repo_id, p.repo_name
),
pr AS (
SELECT 
p.repo_id repo_id, 
p.repo_name repo_name,
count(1) number_of_pr
FROM project_df as p
JOIN pr_df pr on p.repo_id = pr.repo_id
GROUP BY p.repo_id, p.repo_name
)
SELECT p.*, 
i.number_of_issues,
f.number_of_unique_forks,
pr.number_of_pr,
s.number_of_stars
FROM project_df p
LEFT JOIN issues i on i.repo_id = p.repo_id
LEFT JOIN forked f on f.repo_id = p.repo_id
LEFT JOIN pr on pr.repo_id = p.repo_id
LEFT JOIN starred s on s.repo_id = p.repo_id
ORDER BY repo_id LIMIT(30)

""").toPandas()


In [20]:
result.orderBy(1)toPandas()

SyntaxError: invalid syntax (<ipython-input-20-2f3cccec48cc>, line 1)